In [ ]:
!unzip Data.zip

In [ ]:
!pip install dataprep
!pip install unsync

In [3]:
import pandas as pd
import numpy as np
import sqlite3
from dataprep.connector import connect
import asyncio
from unsync import unsync
import matplotlib.pyplot as plt
from numpy import arange
import seaborn as sns

from IPython.core.display import clear_output
from time import sleep, time
from random import randint

import datetime
from geopy.geocoders import Nominatim
import geopy

In [4]:
db_conn = sqlite3.connect("Data/FPA_FOD_20170508.sqlite")
wildfire = pd.read_sql_query ("select * from Fires where FIRE_YEAR>=2010", db_conn)
wildfire

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1067488,1302561,CDF_2010_56_2229_001566,NONFED,ST-CACDF,ST/C&L,USCABEU,San Benito-Monterey Unit,CABEU,CDF - San Benito-Monterey Unit,...,C,36.766944,-121.303056,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xc0\x07\x1bCeS^\xc0...
1,1067489,1302566,CDF_2010_56_2229_001806,NONFED,ST-CACDF,ST/C&L,USCABEU,San Benito-Monterey Unit,CABEU,CDF - San Benito-Monterey Unit,...,B,36.776944,-121.311111,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xd0\xb7\x92>\xe9S^\...
2,1067490,1302568,CDF_2010_56_2229_001857,NONFED,ST-CACDF,ST/C&L,USCABEU,San Benito-Monterey Unit,CABEU,CDF - San Benito-Monterey Unit,...,A,36.856111,-121.381111,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xe4e\xda\x1fdX^\xc0...
3,1067491,1302570,CDF_2010_56_2229_001897,NONFED,ST-CACDF,ST/C&L,USCABEU,San Benito-Monterey Unit,CABEU,CDF - San Benito-Monterey Unit,...,A,36.818056,-121.391111,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00T\xa3\xe4\xf6\x07Y^\...
4,1067492,1302576,CDF_2010_56_2229_002326,NONFED,ST-CACDF,ST/C&L,USCABEU,San Benito-Monterey Unit,CABEU,CDF - San Benito-Monterey Unit,...,B,36.883056,-121.561944,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x04\xfe\xcf\xe5\xf6...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450229,1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,...,A,40.481637,-122.389375,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\xb8\x1e\x85\xeb\x9...
450230,1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
450231,1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
450232,1880464,300348377,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,B,37.672235,-120.898356,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00x\xba_\xaa~9^\xc0\xb...


In [5]:
def get_datetime(row):
  year = row["FIRE_YEAR"]
  doy = row["DISCOVERY_DOY"]
  return datetime.datetime.strptime(f"{year} {doy}", '%Y %j')

wildfire["datetime"] = wildfire.apply(lambda row: get_datetime(row), axis=1)

In [6]:
pd1 = wildfire.groupby(["LATITUDE", "LONGITUDE", "FIRE_YEAR", "DISCOVERY_DOY"]).size().reset_index(name="size")
pd1

,LATITUDE,LONGITUDE,FIRE_YEAR,DISCOVERY_DOY,size
0,17.944924,-67.180858,2014,170,1
1,17.951389,-67.195556,2014,84,1
2,17.951940,-67.212500,2014,233,1
3,17.953889,-67.200833,2014,158,1
4,17.955278,-67.135278,2011,94,1
...,...,...,...,...,...
447273,69.336700,-151.481700,2010,162,1
447274,69.452500,-154.122200,2012,170,1
447275,70.137800,-151.123600,2012,171,1
447276,70.138100,-150.628100,2012,171,1


In [7]:
def get_round_lat(row):
  lat = row["LATITUDE"]
  return round(lat, 0)

def get_round_lon(row):
  lon = row["LONGITUDE"]
  return round(lon, 0)

In [8]:
wildfire["lat"] = wildfire.apply(lambda row: get_round_lat(row), axis=1)
wildfire["lon"] = wildfire.apply(lambda row: get_round_lon(row), axis=1)

In [9]:
pd2 = wildfire.groupby(["lat", "lon", "FIRE_YEAR"]).size().reset_index(name="size")
pd2

,lat,lon,FIRE_YEAR,size
0,18.0,-67.0,2010,1123
1,18.0,-67.0,2011,1493
2,18.0,-67.0,2012,7
3,18.0,-67.0,2013,19
4,18.0,-67.0,2014,39
...,...,...,...,...
5885,69.0,-154.0,2012,1
5886,69.0,-153.0,2010,1
5887,69.0,-151.0,2010,1
5888,70.0,-151.0,2012,2


In [10]:
api_conn = connect("./weatherConfig", _auth={"access_token":"cf06ad2b660347ac8a0295951197e8d3"}) # connect to OpenWeatherMap
# cf06ad2b660347ac8a0295951197e8d3
# 1e3ebb7484174a3ea4a64acecabe304a #
# 6b51a4459fa74430853c7827269c9ca2
# 38e1e3ea5f594166bcd15f936508a930
# d7843e50413c42fda811c8f5e18d49d1
# f5d7fec249d544ba85f082e3f3c0991a

In [11]:
@unsync
async def get_history_weather(api_conn, lat, lon, start_date, end_date):
  df = await api_conn.query("weather_history", lat=lat, lon=lon, start_date=start_date, end_date=end_date)
  return df

In [12]:
weather = pd.DataFrame(columns=["rh", "max_wind_spd_ts", "t_ghi", "max_wind_spd", "solar_rad", "wind_gust_spd", "max_temp_ts", "min_temp_ts",
                                "clouds", "max_dni", "precip_gpm", "wind_spd", "slp", "ts", "max_ghi", "temp", "pres", "dni", "dewpt", "snow",
                                "dhi", "precip", "wind_dir", "max_dhi", "ghi", "max_temp", "t_dni", "max_uv", "t_dhi", "datetime", "t_solar_rad",
                                "min_temp", "max_wind_dir", "snow_depth", "lat", "lon"])
frames = []
start_time = time()

In [13]:
def get_weather_pre_row(row):
  lat = row["lat"]
  lon = row["lon"]
  year = int(row["FIRE_YEAR"])
  start_date = f"{year}-01-01"
  end_date = f"{year + 1}-01-01"
  temp = get_history_weather(api_conn, lat, lon, start_date, end_date).result()
  print(f"temp length is {temp.shape[0]}")
  temp["lat"] = lat
  temp["lon"] = lon
  frames.append(temp)
  print(f"current frame length is {len(frames)}")
  sleep(randint(8,15))
  # Monitor the requests
  requests = len(frames)
  elapsed_time = time() - start_time
  print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
  clear_output(wait = True)

In [ ]:
pd2.apply(lambda row: get_weather_pre_row(row), axis=1)

temp length is 365
current frame length is 9


In [ ]:
weather = pd.concat(frames)
weather

,rh,max_wind_spd_ts,t_ghi,max_wind_spd,solar_rad,wind_gust_spd,max_temp_ts,min_temp_ts,clouds,max_dni,...,t_dni,max_uv,t_dhi,datetime,t_solar_rad,min_temp,max_wind_dir,snow_depth,lat,lon
0,74,1262368800,5349,4,194,4,1262372400,1262347200,33,922,...,7958,4,977,2010-01-01,4657,22,114,0,18.0,-67.0
1,73,1262451600,5359,4,221,4,1262466000,1262430000,21,922,...,7968,6,979,2010-01-02,5322,24,117,0,18.0,-67.0
2,73,1262538000,5370,3,221,3,1262545200,1262520000,17,922,...,7979,5,981,2010-01-03,5308,22,111,0,18.0,-67.0
3,73,1262653200,5383,3,219,3,1262635200,1262602800,30,923,...,7992,6,983,2010-01-04,5257,22,123,0,18.0,-67.0
4,73,1262714400,5396,4,224,4,1262718000,1262664000,20,923,...,8005,6,985,2010-01-05,5377,22,131,0,18.0,-67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,89,1293465600,5310,3,42,3,1293465600,1293451200,98,921,...,7973,2,977,2010-12-27,1008,23,230,0,18.0,-66.0
361,85,1293548400,5317,6,146,6,1293544800,1293523200,83,921,...,7971,3,977,2010-12-28,3522,21,292,0,18.0,-66.0
362,77,1293638400,5324,3,187,3,1293638400,1293620400,48,922,...,7971,4,977,2010-12-29,4495,22,213,0,18.0,-66.0
363,72,1293739200,5332,6,172,6,1293728400,1293710400,59,922,...,7972,4,977,2010-12-30,4142,22,194,0,18.0,-66.0


In [ ]:
weather.to_csv("weather.csv")